# Building your own commodity index

A market index is a hypothetical portfolio of investment holdings that represents a segment of the financial market. The calculation of the index value comes from the prices of the underlying holdings. Some indexes have values based on market-cap weighting, revenue-weighting, float-weighting, and fundamental-weighting. Weighting is a method of adjusting the individual impact of items in an index. [Source](https://www.investopedia.com/terms/m/marketindex.asp)

## Price based index

Price of Commodity A - 150<br>

Price of Commodity B - 50<br>

When an index is created, a base value is chosen e.g 100<br>

The base value is chosen inorder to calculate the index divisor<br>

Index divisor: (150 + 50) / 100 = 2<br>
Index value: 200 (market cap) / 2 (Index divisor) = 100<br>
    
Every other change in the market cap, we divide by the base value of 2.<br>

[Calculating Index Values](https://www.ftserussell.com/education-center/calculating-index-values)

## A Lazy Commodities Index

In terms of commodities:<br>
* We get the value of each commodity on the market.
* We add their prices
* We determine the index divisor.
* We divide total sum of the commodity prices by the index divisor.

### Stages

**closing prices are used**
1. Get the prices of the commodities by sector e.g energy, metals, agriculture.
2. Add the prices of the commodities.<br>
    2.1 Change cents/pence,etc based commodities to dollar based commodities.<br>
    2.2 Change non US commodities to US commodities.<br>
3. Add the prices. 
4. Divide them by a base value of 100 to get divisor.
4. Back test to see if the index follows the commodity prices, by showing on graphs

In [107]:
import tradingeconomics as te
te.login('jupyter:jupyter')

'You are logged in as a jupyter:jupyter'

In [95]:
# Get commodities data
data = te.getMarketsData(marketsField = 'commodities', output_type = 'df')

data['Group'].unique()

array(['Energy', 'Metals', 'Agricultural', 'Index', 'Industrial',
       'Livestock'], dtype=object)

In [98]:
commodities_group = data.query(f'Group == "Metals"') # Choose any commodity.
commodities_group[['Name', 'Close', 'Group', 'unit']]

,Name,Close,Group,unit
5,Gold,1742.6500,Metals,USD/t.oz
6,Silver,19.3515,Metals,USD/t.oz
7,Copper,3.5500,Metals,USD/Lbs
11,Steel,4220.0000,Metals,CNY/T
12,Iron Ore,114.5000,Metals,USD/T
14,Lithium,475500.0000,Metals,CNY/T
23,Platinum,866.2577,Metals,USD/t.oz
59,Titanium,13.8800,Metals,USD/KG
78,HRC Steel,925.0000,Metals,USD/T


##### Non US pegged commodities

In [99]:
non_US_commodities = energy_commodities[~energy_commodities['unit'].str.contains("USD", regex=True)]
non_US_commodities[['Name', 'Close', 'unit']]

,Name,Close,unit
11,Steel,4220.0,CNY/T
14,Lithium,475500.0,CNY/T


In [100]:
# Change cents/pence to dollar/pound
for index, row in energy_commodities.iterrows():
    
    # change pence -> pound & cents to dollar
    # TODO: Use Dataframe.apply function
    if energy_commodities.loc[index, 'unit'].split('/')[0] == 'GBp' or energy_commodities.loc[index, 'unit'].split('/')[0] == 'USd':
        energy_commodities.loc[index, 'Close'] = energy_commodities.loc[index, 'Close'] / 100
        # Change the unit field to reflect conversion


In [101]:
energy_commodities

,Symbol,Ticker,Name,Country,Date,Last,Close,CloseDate,Group,URL,...,YTDPercentualChange,yesterday,lastWeek,lastMonth,lastYear,startYear,decimals,unit,frequency,LastUpdate
5,XAUUSD:CUR,GC1,Gold,commodity,2022-07-07T13:30:00,1742.6500,1742.6500,2022-07-07T13:30:00,Metals,/commodity/gold,...,-4.6894,1738.3035,1806.8870,1853.2550,1802.5550,1828.3903,2.0,USD/t.oz,Live,2022-07-07T13:30:00
6,XAGUSD:CUR,SI1,Silver,commodity,2022-07-07T13:31:00,19.3515,19.3515,2022-07-07T13:31:00,Metals,/commodity/silver,...,-16.8318,19.1811,20.2465,22.0331,25.9055,23.2679,3.0,USD/t.oz,Live,2022-07-07T13:31:00
7,HG1:COM,HG1,Copper,commodity,2022-07-07T13:30:00,3.5500,3.5500,2022-07-07T13:30:00,Metals,/commodity/copper,...,-20.4660,3.4180,3.7145,4.4545,4.2715,4.4635,4.0,USD/Lbs,Live,2022-07-07T13:30:00
11,JBP:COM,Steel,Steel,commodity,2022-07-07T00:00:00,4220.0000,4220.0000,2022-07-07T00:00:00,Metals,/commodity/steel,...,-7.2120,4220.0000,4501.0000,4600.0000,4950.0000,4548.0000,2.0,CNY/T,Delayed,2022-07-07T12:52:00
12,SCO:COM,IRONORE,Iron Ore,commodity,2022-07-06T00:00:00,114.5000,114.5000,2022-07-06T00:00:00,Metals,/commodity/iron-ore,...,-1.2931,114.5000,124.5000,146.5000,222.5000,116.0000,2.0,USD/T,daily,2022-07-07T13:02:00
14,LC:COM,LITHIUM,Lithium,commodity,2022-07-07T00:00:00,475500.0000,475500.0000,2022-07-07T00:00:00,Metals,/commodity/lithium,...,71.3514,475500.0000,475500.0000,474500.0000,89000.0000,277500.0000,0.0,CNY/T,Daily,2022-07-07T12:00:00
23,XPTUSD:CUR,PL1,Platinum,commodity,2022-07-07T13:30:00,866.2577,866.2577,2022-07-07T13:30:00,Metals,/commodity/platinum,...,-10.0031,855.8866,893.8543,1005.9331,1075.5457,962.5414,2.0,USD/t.oz,Delayed,2022-07-07T13:30:00
59,TTSG:COM,TTSG,Titanium,Commodity,2022-07-06T00:00:00,13.8800,13.8800,2022-07-06T00:00:00,Metals,/commodity/titanium,...,95.4930,13.8800,13.8800,13.8800,7.2500,7.1000,2.0,USD/KG,Daily,2022-07-07T13:02:00
78,HRC:COM,HRC,HRC Steel,Commodity,2022-07-07T12:11:00,925.0000,925.0000,2022-07-07T00:00:00,Metals,/commodity/hrc-steel,...,-35.5401,923.0000,930.0000,1150.0000,1780.0000,1435.0000,2.0,USD/T,Delayed,2022-07-07T12:52:00


In [103]:
for index, row in non_US_commodities.iterrows():
    
    # get the base currency for the commodity
    base_currency = str(row['unit']).split('/')[0]
    
    curr_cross = te.getCurrencyCross(cross = base_currency, output_type = 'df')
    curr_pair = base_currency.upper() + 'USD'
    
    curr_usd = curr_cross.loc[curr_cross['Name'] == curr_pair]
    if len(curr_usd) != 0:

        print("1 " + base_currency + " : " + str(curr_usd['Close'][0]) + " USD")
        val = energy_commodities.loc[index, 'Close'] * curr_usd['Close'][0]
        print('1 ' + str(row['unit']).split('/')[1] + ' is ' + str(val) + ' USD')
        
    
    elif len(curr_usd) == 0:
        
        curr_pair = 'USD' + base_currency.upper()
        
        curr_usd = curr_cross.loc[curr_cross['Name'] == curr_pair]
        val = energy_commodities.loc[index, 'Close'] / curr_usd['Close'][0]
        
        print("1 USD" + " : " + str(curr_usd['Close'][0]) + " " + base_currency)
    
    print(val)
    energy_commodities.loc[index, 'Close'] = val
    energy_commodities.loc[index, 'unit'] = 'USD/' + str(row['unit']).split('/')[1]

energy_commodities

1 USD : 6.70058 CNY
629.7962265953096
1 USD : 6.70058 CNY
70964.00610096438


,Symbol,Ticker,Name,Country,Date,Last,Close,CloseDate,Group,URL,...,YTDPercentualChange,yesterday,lastWeek,lastMonth,lastYear,startYear,decimals,unit,frequency,LastUpdate
5,XAUUSD:CUR,GC1,Gold,commodity,2022-07-07T13:30:00,1742.6500,1742.650000,2022-07-07T13:30:00,Metals,/commodity/gold,...,-4.6894,1738.3035,1806.8870,1853.2550,1802.5550,1828.3903,2.0,USD/t.oz,Live,2022-07-07T13:30:00
6,XAGUSD:CUR,SI1,Silver,commodity,2022-07-07T13:31:00,19.3515,19.351500,2022-07-07T13:31:00,Metals,/commodity/silver,...,-16.8318,19.1811,20.2465,22.0331,25.9055,23.2679,3.0,USD/t.oz,Live,2022-07-07T13:31:00
7,HG1:COM,HG1,Copper,commodity,2022-07-07T13:30:00,3.5500,3.550000,2022-07-07T13:30:00,Metals,/commodity/copper,...,-20.4660,3.4180,3.7145,4.4545,4.2715,4.4635,4.0,USD/Lbs,Live,2022-07-07T13:30:00
11,JBP:COM,Steel,Steel,commodity,2022-07-07T00:00:00,4220.0000,629.796227,2022-07-07T00:00:00,Metals,/commodity/steel,...,-7.2120,4220.0000,4501.0000,4600.0000,4950.0000,4548.0000,2.0,USD/T,Delayed,2022-07-07T12:52:00
12,SCO:COM,IRONORE,Iron Ore,commodity,2022-07-06T00:00:00,114.5000,114.500000,2022-07-06T00:00:00,Metals,/commodity/iron-ore,...,-1.2931,114.5000,124.5000,146.5000,222.5000,116.0000,2.0,USD/T,daily,2022-07-07T13:02:00
14,LC:COM,LITHIUM,Lithium,commodity,2022-07-07T00:00:00,475500.0000,70964.006101,2022-07-07T00:00:00,Metals,/commodity/lithium,...,71.3514,475500.0000,475500.0000,474500.0000,89000.0000,277500.0000,0.0,USD/T,Daily,2022-07-07T12:00:00
23,XPTUSD:CUR,PL1,Platinum,commodity,2022-07-07T13:30:00,866.2577,866.257700,2022-07-07T13:30:00,Metals,/commodity/platinum,...,-10.0031,855.8866,893.8543,1005.9331,1075.5457,962.5414,2.0,USD/t.oz,Delayed,2022-07-07T13:30:00
59,TTSG:COM,TTSG,Titanium,Commodity,2022-07-06T00:00:00,13.8800,13.880000,2022-07-06T00:00:00,Metals,/commodity/titanium,...,95.4930,13.8800,13.8800,13.8800,7.2500,7.1000,2.0,USD/KG,Daily,2022-07-07T13:02:00
78,HRC:COM,HRC,HRC Steel,Commodity,2022-07-07T12:11:00,925.0000,925.000000,2022-07-07T00:00:00,Metals,/commodity/hrc-steel,...,-35.5401,923.0000,930.0000,1150.0000,1780.0000,1435.0000,2.0,USD/T,Delayed,2022-07-07T12:52:00


In [104]:
# Sum of all the prices for commodities
total_value = energy_commodities['Close'].sum()
total_value

75278.9915275597

### Get divisor for the index

In [105]:
# Choosing a base value for 100
base_value = 2000
index_divisor = total_value / base_value
index_divisor

37.63949576377985

In [106]:
index_value = total_value / base_value
index_value

37.63949576377985

In [93]:
energy_commodities

,Symbol,Ticker,Name,Country,Date,Last,Close,CloseDate,Group,URL,...,YTDPercentualChange,yesterday,lastWeek,lastMonth,lastYear,startYear,decimals,unit,frequency,LastUpdate
0,CL1:COM,CL1,Crude Oil,commodity,2022-07-07T13:08:00,100.9400,100.940000,2022-07-07T13:08:00,Energy,/commodity/crude-oil,...,34.8024,98.5300,105.7600,119.7800,72.9400,74.8800,3.0,USD/Bbl,Live,2022-07-07T13:08:00
1,CO1:COM,COG1,Brent,commodity,2022-07-07T13:08:00,102.5200,102.520000,2022-07-07T13:08:00,Energy,/commodity/brent-crude-oil,...,32.5404,100.6900,109.0300,120.8900,74.1200,77.3500,2.0,USD/Bbl,Live,2022-07-07T13:08:00
2,NG1:COM,NG1,Natural gas,commodity,2022-07-07T13:08:00,5.8150,5.815000,2022-07-07T13:08:00,Energy,/commodity/natural-gas,...,55.8981,5.5100,5.4240,8.6790,3.6880,3.7300,4.0,USD/MMBtu,Live,2022-07-07T13:08:00
3,XB1:COM,XB1,Gasoline,commodity,2022-07-07T13:08:00,3.3137,3.313700,2022-07-07T13:08:00,Energy,/commodity/gasoline,...,48.6898,3.2366,3.5363,4.0425,2.2552,2.2286,4.0,USD/Gal,Live,2022-07-07T13:08:00
4,HO1:COM,HO1,Heating Oil,commodity,2022-07-07T13:08:00,3.4813,3.481300,2022-07-07T13:08:00,Energy,/commodity/heating-oil,...,50.8101,3.4106,3.8305,4.2198,2.1204,2.3084,4.0,USD/Gal,Live,2022-07-07T13:08:00
10,XAL1:COM,Coal,Coal,commodity,2022-07-06T00:00:00,399.7000,399.700000,2022-07-06T00:00:00,Energy,/commodity/coal,...,135.6722,397.6000,380.0000,403.0000,142.2500,169.6000,2.0,USD/T,Delayed,2022-07-07T12:52:00
15,NGEU:COM,NGEU,TTF Gas,commodity,2022-07-07T12:53:00,182.6400,185.828894,2022-07-07T12:53:00,Energy,/commodity/eu-natural-gas,...,159.6420,170.9990,144.5140,79.4070,33.3120,70.3430,2.0,USD/MWh,Delayed,2022-07-07T13:08:00
16,NGUK:COM,NGUK,UK Gas,commodity,2022-07-07T12:55:00,312.6600,3.739038,2022-07-07T12:55:00,Energy,/commodity/uk-natural-gas,...,83.2278,290.0100,248.3000,129.8900,82.1500,170.6400,4.0,USD/thm,Delayed,2022-07-07T13:05:00
21,DL1:COM,DL1,Ethanol,commodity,2022-07-06T00:00:00,2.3900,2.390000,2022-07-06T00:00:00,Energy,/commodity/ethanol,...,-20.3333,2.4275,2.7975,2.8600,2.2400,3.0000,4.0,USD/Gal,daily,2022-07-07T12:51:00
22,MOB:COM,NAPHTHA,Naphtha,commodity,2022-07-06T00:00:00,787.4820,787.482000,2022-07-06T00:00:00,Energy,/commodity/naphtha,...,12.8120,777.0250,801.1740,855.7530,656.8710,698.0480,2.0,USD/T,daily,2022-07-07T12:52:00
